In [2]:
import cv2
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
from torchvision import transforms
from PIL import Image
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F

d:\Code\Python Code\face_recognition\FR_VM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Testing

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=False, device=device, thresholds=[0.5, 0.5, 0.5])
model = InceptionResnetV1(pretrained="vggface2", classify=False).to(device)
model.eval()

In [56]:
face_vectors = torch.load("./face_embeddings_vggface2.pt") # a list of face embeddings
face_names = np.load("./face_names_vggface2.npy") # a list of names corresponding to the embeddings

In [59]:
def cosine_similarity(vec1, vec2):
    return F.cosine_similarity(vec1, vec2, dim=0).item()

image = Image.open("./dataset/test_image/Aaron_peirsol-2.jpg").convert("RGB")
face_detected = mtcnn(image)

similarities = []

if face_detected is not None:
    feature_vector = model(face_detected.unsqueeze(0).to(device))[0] # [1, 512]
    for i, face_vector in enumerate(face_vectors):
        sim = cosine_similarity(feature_vector, face_vector)
        if sim > 0.5:
            similarities.append((face_names[i], sim))
    if similarities:
        similarities.sort(key=lambda x: x[1], reverse=True)
        print(f"Top matches for the detected face:")
        for name, sim in similarities[:5]:
            print(f"Name: {name}, Similarity: {sim:.4f}")
else:
    print("No face detected in the image.")

Top matches for the detected face:
Name: Aaron_Peirsol, Similarity: 0.8256
Name: German_Khan, Similarity: 0.6118
Name: Greg_Owen, Similarity: 0.5431
Name: Bob_Stoops, Similarity: 0.5336
Name: Clint_Howard, Similarity: 0.5035


## Modify the ways to extract feature vector

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=False, device=device, thresholds=[0.6, 0.7, 0.9])
model = InceptionResnetV1(pretrained="casia-webface", classify=False).to(device)
model.eval()

In [49]:
img_path = "./dataset/raw/lfw-deepfunneled/Aaron_Peirsol"
# save_path = "./dataset/cropped_images/cropped_Aaron_Guiel.jpg"
image_files = [os.path.join(img_path, f) for f in os.listdir(img_path)]
print(image_files)

['./dataset/raw/lfw-deepfunneled/Aaron_Peirsol\\Aaron_Peirsol_0001.jpg', './dataset/raw/lfw-deepfunneled/Aaron_Peirsol\\Aaron_Peirsol_0002.jpg', './dataset/raw/lfw-deepfunneled/Aaron_Peirsol\\Aaron_Peirsol_0003.jpg', './dataset/raw/lfw-deepfunneled/Aaron_Peirsol\\Aaron_Peirsol_0004.jpg']


In [54]:
embeddings = []
for img_file in image_files:
    img = Image.open(img_file).convert("RGB")
    face = mtcnn(img)
    print(face.shape)
    if face is None:
        continue
    embedding = model(face.unsqueeze(0).to(device))
    embeddings.append(embedding)

torch.Size([3, 160, 160])
torch.Size([3, 160, 160])
torch.Size([3, 160, 160])
torch.Size([3, 160, 160])


In [ ]:
embeddings_tensor = torch.cat(embeddings, dim=0)
mean_embedding = torch.mean(embeddings_tensor, dim=0)
print(mean_embedding)
mean_embedding_norm = torch.nn.functional.normalize(mean_embedding, p=2, dim=0)
print(mean_embedding_norm)

In [38]:
test_img = "./dataset/test_image/Aaron_peirsol-2.jpg"
img = Image.open(test_img)
face = mtcnn(img, save_path="./dataset/cropped_images/cropped_Aaron_peirsol.jpg")
vector = model(face.unsqueeze(0).to(device))
print(vector.shape)
vector_norm = torch.nn.functional.normalize(vector, p=2, dim=1)
print(vector_norm.shape)

torch.Size([1, 512])
torch.Size([1, 512])


In [43]:
def cosine_similarity(vec1, vec2):
    return F.cosine_similarity(vec1, vec2, dim=0).item()

similarity = cosine_similarity(mean_embedding_norm, vector_norm[0])
print(f"Cosine similarity between mean embedding and test image: {similarity}")

Cosine similarity between mean embedding and test image: 0.8256192207336426
